In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split

In [2]:
PATH = "data/"
device = "cuda"
seed = 42
file_name = PATH + "train.csv"

In [3]:
class MNIST(Dataset):
    def __init__(self, X, y):
        self.y = torch.from_numpy(y).long().to(device)
        X = X.reshape(X.shape[0], 1, 28, 28)
        X = X / 255.0
        self.X = torch.from_numpy(X).float().to(device)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return (self.X[idx,:,:,:], self.y[idx])

In [4]:
df = pd.read_csv(file_name)
y = df["label"].values
X = df.drop("label", axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

In [5]:
batch_size = 256
train_dataset = MNIST(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

test_dataset = MNIST(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# MobileNet V1

![title](data/DSConvolution.png)

In [6]:
class BnLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0, groups=1):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride,
                              padding=padding, groups=groups, bias=False)
        
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        return x

In [7]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        # Depthwise convolution
        self.bn_layer_3x3 = BnLayer(in_channels, in_channels, kernel_size=3, stride=stride, groups=in_channels)

        # Pointwise convolution
        self.bn_layer_1x1 = BnLayer(in_channels, out_channels, kernel_size=1, stride=1)

    def forward(self, x):
        x = self.bn_layer_3x3(x)
        x = self.bn_layer_1x1(x)
        return x

In [8]:
class MobileNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.init_layer = BnLayer(1, 16, kernel_size=3, stride=2)
        self.block_1 = Block(16, 32, stride=2)
        self.block_2 = Block(32, 64, stride=2)

        self.dropout = nn.Dropout(p=0.5)
        self.out = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.init_layer(x)
        x = self.block_1(x)
        x = self.block_2(x)

        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.out(x)
        return F.log_softmax(x, dim=-1)

In [9]:
n_epochs = 50
lr = 1e-3
weight_decay = 1e-5

torch.manual_seed(seed)
model = MobileNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
lr_sceduler = StepLR(optimizer, gamma=0.5, step_size=10)

In [10]:
torch.manual_seed(seed)
for epoch in range(0, n_epochs):
    lr_sceduler.step()
    model.train()
    train_loss = 0
    for _, (x, y) in enumerate(train_loader):
            model.zero_grad()
            output = model(x)
            batch_loss = F.nll_loss(output, y)

            batch_loss.backward()
            optimizer.step()
            train_loss += batch_loss.cpu().detach().numpy() / x.shape[0]

    train_loss = np.round(train_loss / len(train_loader), 6)

    model.eval()
    test_loss = 0
    test_acc = 0
    for _, (x, y) in enumerate(test_loader):
        output = model(x)
        batch_loss = F.nll_loss(output, y)
        test_loss += batch_loss.cpu().detach().numpy() / x.shape[0]

        y_hat = torch.argmax(output, dim=1)
        acc = (y_hat == y).sum().float() / x.shape[0]
        acc = acc.cpu().detach().numpy()
        test_acc += acc

    test_loss = np.round(test_loss / len(test_loader), 6)
    test_acc = np.round(test_acc / len(test_loader), 4)

    if epoch % 5 == 0:
        print("Epoch {}: train loss {}, test loss {}, test accuracy {}".format(epoch,
                                                                               train_loss,
                                                                               test_loss,
                                                                               test_acc))

Epoch 0: train loss 0.007893, test loss 0.00666, test accuracy 0.5632
Epoch 5: train loss 0.001544, test loss 0.001097, test accuracy 0.9318
Epoch 10: train loss 0.001016, test loss 0.000732, test accuracy 0.9507
Epoch 15: train loss 0.000912, test loss 0.000645, test accuracy 0.9546
Epoch 20: train loss 0.000812, test loss 0.0006, test accuracy 0.9591
Epoch 25: train loss 0.000778, test loss 0.000582, test accuracy 0.9597
Epoch 30: train loss 0.000754, test loss 0.000568, test accuracy 0.9599
Epoch 35: train loss 0.000736, test loss 0.000561, test accuracy 0.9603
Epoch 40: train loss 0.000727, test loss 0.000556, test accuracy 0.9613
Epoch 45: train loss 0.000714, test loss 0.000551, test accuracy 0.9619
